### Libraries

In [ ]:
import numpy as np
from sympy import symbols
import math
import csv
from tabulate import tabulate
import sympy as sp

### Accessing Sheet Data

In [ ]:
# Define a class for nodes
class Node:
    def __init__(self, support_type, coordinates, degrees_of_freedom, reaction_forces_exist):
        self.support_type = support_type
        self.coordinates = coordinates
        self.degrees_of_freedom = degrees_of_freedom
        self.reaction_forces_exist = reaction_forces_exist

# Define a class for members
class Member:
    def __init__(self, node1_id, node2_id, cross_section_area, modulus_of_elasticity, moment_of_inertia, length, angle, loads_info):
        self.node1_id = node1_id
        self.node2_id = node2_id
        self.cross_section_area = cross_section_area
        self.modulus_of_elasticity = modulus_of_elasticity
        self.moment_of_inertia = moment_of_inertia
        self.length = length
        self.angle = angle
        self.loads_info = loads_info

In [ ]:
# Function to determine degrees of freedom based on support type
def determine_degrees_of_freedom(support_type):
    u, v, theta = 0, 0, 0
    if support_type == 'fixed':
        u, v, theta = 0, 0, 0
    elif support_type == 'pinned':
        theta = 1
    elif support_type == 'hroller':
        u, theta = 1, 1
    elif support_type == 'vroller':
        v, theta = 1, 1
    elif support_type == 'free':
        u, v, theta = 1, 1, 1
    return u, v, theta

# Function to determine existence of external forces and moments based on support type
def determine_unknown_forces(support_type):
    H_exists, V_exists, M_exists = False, False, False

    if support_type == 'fixed':
        H_exists, V_exists, M_exists = True, True, True
    elif support_type == 'pinned':
        H_exists, V_exists = True, True
    elif support_type == 'hroller':
        V_exists = True
    elif support_type == 'vroller':
        H_exists = True
    elif support_type == 'free':
        H_exists, V_exists, M_exists = False, False, False

    return H_exists, V_exists, M_exists

In [ ]:
def import_nodes_from_csv(filename):
    nodes = {}
    external_forces = {}
    settlements = {}
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            node_id = int(row['node_id'])
            support_type = row['support_type'].strip().lower()
            coordinates = [float(row['x']), float(row['y'])]
            degrees_of_freedom = determine_degrees_of_freedom(support_type)
            Hu, Vu, Mu = determine_unknown_forces(support_type)
            node = Node(support_type, coordinates, degrees_of_freedom, {'H': Hu, 'V': Vu, 'M': Mu})
            nodes[node_id] = node
            # External forces and settlements
            external_forces[node_id] = (
                float(row.get('external_H', 0)),
                float(row.get('external_V', 0)),
                float(row.get('external_M', 0))
            )
            settlements[node_id] = (
                float(row.get('settlement_u', 0)),
                float(row.get('settlement_v', 0)),
                float(row.get('settlement_theta', 0))
            )
    return nodes, external_forces, settlements

def parse_loads(loads_str):
    loads = []
    if not loads_str:
        return loads
    for load_item in loads_str.split('|'):
        parts = load_item.strip().split(';')
        load_type = parts[0].strip()

        # Split comma-separated distance values
        load_values = []
        for val in parts[1:]:
            # Remove commas if present (e.g., "2,3" → "2 3")
            cleaned_val = val.replace(',', ' ').split()
            load_values.extend([float(x) for x in cleaned_val])

        # Organize based on load type
        if load_type == 'point_load':
            # Format: point_load;P;a;b → [5000, 2, 3]
            load_value = [load_values[0]]
            load_distance = load_values[1:3]
        elif load_type == 'uniform_distributed_load':
            # Format: uniform_distributed_load;w;a;b → [2000, 1, 4]
            load_value = [load_values[0]]
            load_distance = load_values[1:3]

        loads.append((load_type, load_value, load_distance))
    return loads

def import_members_from_csv(filename, nodes):
    members = {}
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            member_id = int(row['member_id'])
            node1_id = int(row['node1'])
            node2_id = int(row['node2'])
            A = float(row['A'])
            E = float(row['E'])
            I = float(row['I'])
            loads_info = parse_loads(row.get('loads', ''))
            # Calculate length and angle
            x1, y1 = nodes[node1_id].coordinates
            x2, y2 = nodes[node2_id].coordinates
            length = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
            angle = math.atan2(y2 - y1, x2 - x1) * 180 / math.pi
            member = Member(node1_id, node2_id, A, E, I, length, angle, loads_info)
            members[member_id] = member
    return members

def take_node_data(num_nodes):
    nodes = {}
    for i in range(num_nodes):
        print(f"\nEnter details for Node {i+1}:")
        support_type = input("Type of Support (fixed, pinned, hroller, vroller, free): ").strip().lower()
        coordinates = input("Coordinates of Node (x, y separated by space): ").strip().split()
        coordinates = [float(coord) for coord in coordinates]
        degrees_of_freedom = determine_degrees_of_freedom(support_type)
        Hu, Vu, Mu = determine_unknown_forces(support_type)
        node = Node(support_type, coordinates, degrees_of_freedom, {'H': Hu, 'V': Vu, 'M': Mu})
        nodes[i+1] = node
    return nodes

# Function to take input for member details
def take_member_data(num_members, nodes):
    members = {}
    for i in range(num_members):
        print(f"\nEnter details for Member {i+1}:")
        node1_id = int(input("Enter ID of first node: "))
        node2_id = int(input("Enter ID of second node: "))
        cross_section_area = float(input("Enter cross-section area of member: "))
        modulus_of_elasticity = float(input("Enter modulus of elasticity of member: "))
        moment_of_inertia = float(input("Enter moment of inertia of member: "))
        num_loads = int(input("Enter the number of loads on this member: "))
        loads_info = []
        for j in range(num_loads):
            load_type = input(f"Enter Load Type {j+1} (point_load, uniform_distributed_load, uniform_varying_load, trapezoidal_load): ").strip().lower()
            load_value = input(f"Enter Load Value {j+1} (separated by space, if none enter 0): ").strip().split()
            load_value = [float(f) for f in load_value]
            load_distance = input(f"Enter Distance of Load {j+1} from Node 1 and Node 2 (separated by space as a b, if none enter 0 0): ").strip().split()
            load_distance = [float(f) for f in load_distance[:2]]  # Take only the first two values
            loads_info.append((load_type, load_value, load_distance))
        length = ((nodes[node2_id].coordinates[0] - nodes[node1_id].coordinates[0])**2 + (nodes[node2_id].coordinates[1] - nodes[node1_id].coordinates[1])**2) ** 0.5
        angle = math.atan2(nodes[node2_id].coordinates[1] - nodes[node1_id].coordinates[1], nodes[node2_id].coordinates[0] - nodes[node1_id].coordinates[0]) * 180 / math.pi
        member = Member(node1_id, node2_id, cross_section_area, modulus_of_elasticity, moment_of_inertia, length, angle, loads_info)
        members[i+1] = member
    return members

nodes_csv = 'nodes.csv'
members_csv = 'members.csv'

nodes_data, external_forces, settlements = import_nodes_from_csv(nodes_csv)
member_data = import_members_from_csv(members_csv, nodes_data)

### Understanding DATA

In [ ]:
# Function to calculate member lengths and angles
def calculate_member_length(n_val, m_val):
    L = []  # List to store member lengths
    th = []  # List to store member angles

    for i in m_val.keys():
        node1 = m_val[i].node1_id
        node2 = m_val[i].node2_id
        x1, y1 = n_val[node1].coordinates
        x2, y2 = n_val[node2].coordinates
        length = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
        angle = math.atan2(y2 - y1, x2 - x1) * 180 / math.pi
        L.append(length)
        th.append(angle)

    return L, th

L_val, th_val = calculate_member_length(nodes_data, member_data)
print("Member Lenghts:")
print(L_val)
print("Member Theta:")
print(th_val)

Member Lenghts:
[9.433981132056603, 4.0]
Member Theta:
[32.005383208083494, 0.0]


In [ ]:
# Function to calculate external force matrix on Nodes
def calculate_external_force_matrix(nodes, external_forces):
    external_force = []
    for node_id, node in nodes.items():
        H = external_forces[node_id][0]
        V = external_forces[node_id][1]
        M = external_forces[node_id][2]
        external_force.extend([H, V, M])
    external_force_matrix = np.array(external_force).reshape(-1, 1)
    print("\nExternal Force Matrix:")
    print(np.array(external_force_matrix))
    return external_force_matrix

external_force_matrix = calculate_external_force_matrix(nodes_data, external_forces)


External Force Matrix:
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


### Matrices - L,T,G

In [ ]:
# Function to calculate local stiffness matrix
def l_stiffness_matrix(A, E, L, I):
    k_local = np.zeros((6, 6))
    k_local[0, 0] = A * E / L
    k_local[0, 3] = -A * E / L
    k_local[1, 1] = 12 * E * I / (L ** 3)
    k_local[1, 2] = 6 * E * I / (L ** 2)
    k_local[1, 4] = -12 * E * I / (L ** 3)
    k_local[1, 5] = 6 * E * I / (L ** 2)
    k_local[2, 1] = 6 * E * I / (L ** 2)
    k_local[2, 2] = 4 * E * I / L
    k_local[2, 4] = -6 * E * I / (L ** 2)
    k_local[2, 5] = 2 * E * I / L
    k_local[3, 3] = A * E / L
    k_local[3, 0] = -A * E / L
    k_local[4, 1] = -12 * E * I / (L ** 3)
    k_local[4, 2] = -6 * E * I / (L ** 2)
    k_local[4, 4] = 12 * E * I / (L ** 3)
    k_local[4, 5] = -6 * E * I / (L ** 2)
    k_local[5, 1] = 6 * E * I / (L ** 2)
    k_local[5, 2] = 2 * E * I / L
    k_local[5, 4] = -6 * E * I / (L ** 2)
    k_local[5, 5] = 4 * E * I / L
    return k_local

local_sf_mat = l_stiffness_matrix(member_data[1].cross_section_area, member_data[1].modulus_of_elasticity, member_data[1].length, member_data[1].moment_of_inertia)
print(local_sf_mat)

[[ 2.17299565e+08  0.00000000e+00  0.00000000e+00 -2.17299565e+08
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.44059152e+04  1.15122472e+05  0.00000000e+00
  -2.44059152e+04  1.15122472e+05]
 [ 0.00000000e+00  1.15122472e+05  7.24042152e+05  0.00000000e+00
  -1.15122472e+05  3.62021076e+05]
 [-2.17299565e+08  0.00000000e+00  0.00000000e+00  2.17299565e+08
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.44059152e+04 -1.15122472e+05  0.00000000e+00
   2.44059152e+04 -1.15122472e+05]
 [ 0.00000000e+00  1.15122472e+05  3.62021076e+05  0.00000000e+00
  -1.15122472e+05  7.24042152e+05]]


In [ ]:
# Function to perform transformation to global coordinate system
def transformation(theta, transpose=False):
    c = np.cos(np.radians(theta))
    s = np.sin(np.radians(theta))
    trans_matrix = np.array([
        [c, -s, 0, 0, 0, 0],
        [s, c, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, c, -s, 0],
        [0, 0, 0, s, c, 0],
        [0, 0, 0, 0, 0, 1]
    ])
    if transpose:
        trans_matrix = np.transpose(trans_matrix)
    return trans_matrix

t_mat = transformation(th_val[0])
print(t_mat)
t_mat = transformation(th_val[1])
print(t_mat)

[[ 0.8479983  -0.52999894  0.          0.          0.          0.        ]
 [ 0.52999894  0.8479983   0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.        ]
 [ 0.          0.          0.          0.8479983  -0.52999894  0.        ]
 [ 0.          0.          0.          0.52999894  0.8479983   0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]
[[ 1. -0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


In [ ]:
# Function to calculate global stiffness matrix
def calculate_global_stiffness_matrix(member_data, nodes):
    num_elements = len(member_data)
    k_local_list = [l_stiffness_matrix(member_data[i].cross_section_area, member_data[i].modulus_of_elasticity, member_data[i].length, member_data[i].moment_of_inertia) for i in range(1, num_elements + 1)]
    num_nodes = num_elements + 1
    num_dofs = 3
    K_global = np.zeros((num_nodes * num_dofs, num_nodes * num_dofs))
    t_matrix_list = [transformation(member_data[i].angle) for i in range(1, num_elements + 1)]
    for i in range(num_elements):
        k_local_transformed = np.dot(np.dot(t_matrix_list[i], k_local_list[i]), np.transpose(t_matrix_list[i]))
        start_index = (i * num_dofs)
        end_index = ((i + 2) * num_dofs)
        for m in range(num_dofs * 2):
            for n in range(num_dofs * 2):
                K_global[start_index + m, start_index + n] += k_local_transformed[m, n]

    return K_global

global_stiffness_matrix = calculate_global_stiffness_matrix(member_data, nodes_data)
print(global_stiffness_matrix)

[[ 1.56267217e+08  9.76517571e+07 -6.10147881e+04 -1.56267217e+08
  -9.76517571e+07 -6.10147881e+04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 9.76517571e+07  6.10567541e+07  9.76236609e+04 -9.76517571e+07
  -6.10567541e+07  9.76236609e+04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-6.10147881e+04  9.76236609e+04  7.24042152e+05  6.10147881e+04
  -9.76236609e+04  3.62021076e+05  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-1.56267217e+08 -9.76517571e+07  6.10147881e+04  6.68767217e+08
   9.76517571e+07  6.10147881e+04 -5.12500000e+08  0.00000000e+00
   0.00000000e+00]
 [-9.76517571e+07 -6.10567541e+07 -9.76236609e+04  9.76517571e+07
   6.13769385e+07  5.42745089e+05  0.00000000e+00 -3.20184375e+05
   6.40368750e+05]
 [-6.10147881e+04  9.76236609e+04  3.62021076e+05  6.10147881e+04
   5.42745089e+05  2.43169215e+06  0.00000000e+00 -6.40368750e+05
   8.53825000e+05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.12500000e+08
   0.00000000e+00  0.0000000

### Other Matrices

In [ ]:
# Function to calculate displacement matrix
def calculate_displacement_matrix(nodes):
    disp_values = []
    for node_id, node in nodes.items():
        u = symbols('Horizontal_Displacement_' + str(node_id)) if node.degrees_of_freedom[0] else 0
        v = symbols('Vertical_Displacement_' + str(node_id)) if node.degrees_of_freedom[1] else 0
        theta = symbols('theta_' + str(node_id)) if node.degrees_of_freedom[2] else 0
        disp_values.extend([u, v, theta])
    disp_matrix = np.array(disp_values).reshape(-1, 1)
    print("\nDisplacement Matrix:")
    print(disp_matrix)
    return disp_matrix

displacement_matrix = calculate_displacement_matrix(nodes_data)


Displacement Matrix:
[[0]
 [0]
 [0]
 [Horizontal_Displacement_2]
 [Vertical_Displacement_2]
 [theta_2]
 [0]
 [0]
 [0]]


In [ ]:
# Function to calculate unknown force matrix i.e. reactions
def calculate_unknown_force_matrix(nodes):
    unknown_force = []
    for node_id, node in nodes.items():
        support_type = node.support_type
        H_exists, V_exists, M_exists = determine_unknown_forces(support_type)

        Hu = symbols('Horizontal_Reaction_' + str(node_id)) if H_exists else 0
        Vu = symbols('Vertical_Reaction_' + str(node_id)) if V_exists else 0
        Mu = symbols('Moment_' + str(node_id)) if M_exists else 0

        unknown_force.extend([Hu, Vu, Mu])

    unknown_force_matrix = np.array(unknown_force).reshape(-1, 1)
    print("Reaction Force Matrix:")
    print(np.array(unknown_force_matrix))
    return unknown_force_matrix

unknown_force_matrix = calculate_unknown_force_matrix(nodes_data)

Reaction Force Matrix:
[[Horizontal_Reaction_1]
 [Vertical_Reaction_1]
 [Moment_1]
 [0]
 [0]
 [0]
 [Horizontal_Reaction_3]
 [Vertical_Reaction_3]
 [Moment_3]]


In [ ]:
#Settlement
def calculate_settlement_matrix(nodes, settlements):
    settlement = []
    for node_id, node in nodes.items():
        u_s = settlements[node_id][0]
        v_s = settlements[node_id][1]
        theta_s = settlements[node_id][2]
        settlement.extend([u_s, v_s, theta_s])
    settlement_matrix = np.array(settlement).reshape(-1, 1)
    return settlement_matrix

settlement_matrix = calculate_settlement_matrix(nodes_data, settlements)
s = settlement_matrix + displacement_matrix

### Results

In [ ]:
# Function to calculate fixed end forces due to point load
def point_load_fixed_end(L, P, a, b):
    F1 = -(P * (b ** 2) * (L + (2 * a))) / (L ** 3)
    F2 = -(P * (a ** 2) * (L + (2 * b))) / (L ** 3)
    M1 = -(P * (b ** 2) * a) / L ** 2
    M2 = (P * (a ** 2) * b) / L ** 2
    return F1, F2, M1, M2

# Function to calculate fixed end forces due to uniform distributed load
def uniform_distributed_load_fixed_end(L, w, a, b):
    c = L-a-b
    d = b+(c/2)
    e = a+c
    F1 = -(w*c*((12*(d**2))-((8*(d**3))/(L)) + ((2*e*(c**2))/(L)) - ((c**3)/(L)) - (c**2)))/(4*(L**2))
    M1 = -(w*c*(((24*(d**3))/(L))-((6*e*(c**2))/(L))+((3*(c**3))/(L))+(4*(c**2))-(24*(d**2))))/(24*L)
    F2 = -(w*c)+((w*c*((12*(d**2))-((8*(d**3))/(L))+((2*e*(c**2))/(L))-((c**3)/(L))-(c**2)))/(4*(L**2)))
    M2 = (w*c*(((24*(d**3))/(L))-((6*e*(c**2))/(L))+((3*(c**3))/(L))+(4*(c**2))-(24*(d**2))))/(24*L)
    return F1, F2, M1, M2

# Function to calculate fixed end forces due to uniform varying load
def uniform_varying_load_fixed_end(L, w1, w2,s1,s3):
    s2 = L-s1-s3

    M2 = -((-1/60)*(w2)*(s2)*((3*(s2**3))+((15*(s2**2)*(s3)))+((10*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((10*(s2**2)*(s1)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)) + ((-1/60)*(w1)*(s2)*((2*(s2**3))+((5*(s2**2)*(s3)))+((20*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((10*(s2**2)*(s1)))+ ((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**2))
    M1 = (((-1/60)*(w1)*(s2)*((3*(s2**3))+((15*(s2**2)*(s1)))+((10*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((10*(s2**2)*(s3)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)) + ((-1/60)*(w2)*(s2)*((2*(s2**3))+((5*(s2**2)*(s1)))+((20*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((10*(s2**2)*(s3)))+ ((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)))
    F2 = -((1/20)*(w1)*(s2)*((3*(s2**3))+((5*(s2**2)*(s3)))+((10*(s1**3)*(1)))+((30*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((15*(s2**2)*(s1)))+((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**3)) + ((1/20)*(w2)*(s2)*((7*(s2**3))+((15*(s2**2)*(s3)))+((10*(s1**3)*(1)))+((30*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((25*(s2**2)*(s1)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**3))
    F1 = -((1/20)*(w2)*(s2)*((3*(s2**3))+((5*(s2**2)*(s1)))+((10*(s3**3)*(1)))+((30*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((15*(s2**2)*(s3)))+((20*(s2)*(s3)*(s1))))/((s1+s2+s3)**3)) + ((1/20)*(w1)*(s2)*((7*(s2**3))+((15*(s2**2)*(s1)))+((10*(s3**3)*(1)))+((30*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((25*(s2**2)*(s3)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**3))
    return F1, F2, M1, M2

# Function to calculate fixed end forces due to trapezoidal load
def trapezoidal_load_fixed_end(L, w1, w2):
    F1 = (w1 + w2) * L / 4
    F2 = (w1 + w2) * L / 4
    M1 = w1 * L ** 2 / 12
    M2 = w2 * L ** 2 / 12
    return F1, F2, M1, M2


In [ ]:
def calculate_global_fixed_end_forces(member_data):
    num_elements = len(member_data)
    no_node = num_elements + 1
    global_fixed_end_forces_matrix = np.zeros((3 * no_node, 1))

    for i, data in member_data.items():
        for load_info in data.loads_info:
            load_type, load_value, load_distance = load_info
            length = data.length
            angle = data.angle

            F1, F2, M1, M2 = 0, 0, 0, 0

            if load_type == 'point_load':
                F1, F2, M1, M2 = point_load_fixed_end(length, load_value[0], load_distance[0], load_distance[1])
            elif load_type == 'uniform_distributed_load':
                F1, F2, M1, M2 = uniform_distributed_load_fixed_end(length, load_value[0],load_distance[0], load_distance[1])
            elif load_type == 'uniform_varying_load':
                F1, F2, M1, M2 = uniform_varying_load_fixed_end(length, load_value[0], load_value[1],load_distance[0], load_distance[1])
            elif load_type == 'trapezoidal_load':
                F1, F2, M1, M2 = trapezoidal_load_fixed_end(length, load_value[0], load_value[1])

            local_fixed = np.zeros((6,1))

            # Assign values to the corresponding positions
            local_fixed[0] = 0
            local_fixed[1] = F1
            local_fixed[2] = M1
            local_fixed[3] = 0
            local_fixed[4] = F2
            local_fixed[5] = M2


# Anticlockwise positive(moment)
            node1_id = data.node1_id
            node2_id = data.node2_id
            t_matrix = [transformation(member_data[i].angle)]
            local_fixed_transformed = np.dot(t_matrix, local_fixed)
            local_fixed_t = local_fixed_transformed.reshape(6, 1)

            # Assemble local forces to global fixed end forces matrix
            global_fixed_end_forces_matrix[(node1_id - 1) * 3:(node1_id - 1) * 3 + 6] += local_fixed_t

    print("\nGlobal Fixed End Forces Matrix:")
    print(np.array(global_fixed_end_forces_matrix) / 1000)
    return global_fixed_end_forces_matrix
global_fixed_end_forces = calculate_global_fixed_end_forces(member_data)


Global Fixed End Forces Matrix:
[[ -75. ]
 [ 120. ]
 [-222.5]
 [ -75. ]
 [ 180. ]
 [ 182.5]
 [   0. ]
 [  60. ]
 [  40. ]]


In [ ]:
result = np.dot(global_stiffness_matrix, s)
result_with_forces = result + global_fixed_end_forces - external_force_matrix - unknown_force_matrix

equations = [i[0] for i in result_with_forces]
variables = set()
for eq in equations:
    variables.update(eq.free_symbols)
x= sp.symbols(' '.join(str(variable) for variable in variables))
solution = sp.solve(equations,x)

# Convert the solution dictionary to a list of lists
solution_table = [["Variable", "Value"]]
for key, value in solution.items():
    try:
        # Check if the key indicates a displacement
        if 'Displacement' in str(key):
            # If it's a displacement, keep the original value
            solution_table.append([key, value])
        # Check if the key indicates a vertical reaction
        elif 'Vertical_Reaction' in str(key):
            # If it's a vertical reaction, multiply the value by -1
            solution_table.append([key, value /1000])
        else:
            # Otherwise, divide the value by 1000
            solution_table.append([key, value / 1000])
    except TypeError:
        # If it's not a number (e.g., a symbolic expression), keep it as is
        solution_table.append([key, value])

# Print the solution table with selective division and sign change
print(tabulate(solution_table, headers="firstrow"))

Variable                            Value
-------------------------  --------------
Horizontal_Displacement_2     0.000587684
Horizontal_Reaction_1       151.188
Horizontal_Reaction_3      -301.188
Moment_1                   -249.067
Moment_3                    -25.5367
Vertical_Displacement_2      -0.00321026
Vertical_Reaction_1         251.361
Vertical_Reaction_3         108.639
theta_2                      -7.43488e-05
